initial imports

In [3]:
from itertools import groupby
import pandas as pd
import numpy as np
import matplotlib.pyplot as pp

define fields

In [96]:
_PART       = 'RECORDING_SESSION_LABEL'
PART        = 'Participant'
_TRIAL      = 'TRIAL_INDEX'
TRIAL       = 'Trial'
_TARGET     = 'AVERAGE_IA_1_SAMPLE_COUNT_%'
TARGET      = 'Target'
_FILL1      = 'AVERAGE_IA_3_SAMPLE_COUNT_%'
FILL1       = 'Filler1'
_COMP       = 'AVERAGE_IA_2_SAMPLE_COUNT_%'
COMP        = 'Competitor'
_FILL2      = 'AVERAGE_IA_4_SAMPLE_COUNT_%'
FILL2       = 'Filler2'
_TIME       = 'BIN_START_TIME'
TIME        = 'Time'
TYPE        = 'type'
VERSION     = 'version'
TARGET_WORD = 'target_word'


define calculated fields

In [5]:
# calculated special
FILLAvg    = 'Filler Average'
NTAvg      = 'Non Target Average'
TCAvg      = 'Target Competitor Average'
TmC        = 'Target minus Competitor'
TmF        = 'Target minus Filler'
TmNT       = 'Target minus Non Target'
TCmF       = 'Target Competitor minus Filler'

define columns to keep when clearing up some elements

In [6]:
KEEP_COLUMNS = [TIME, PART, TRIAL, TYPE, VERSION, TARGET, COMP, FILL1, FILL2, TARGET_WORD,
                FILLAvg, NTAvg, TCAvg, TmC, TmF, TmNT, TCmF]

define some colors and names for plotting

In [7]:

TARGET_COLOR  = 'red'
COMP_COLOR    = 'blue'
NTargetCOLOR  = 'green'
color_dict={COMP: COMP_COLOR, TARGET: TARGET_COLOR, NTAvg: NTargetCOLOR, FILLAvg: NTargetCOLOR, TCAvg: TARGET_COLOR}


set files names

In [48]:
F_OLD_DATA = 'data/240621_old_time_course_binning.xls.csv'
F_OLD_TOUCH_DATA ='data/240621_old_message_report.xls.csv'
F_YOUNG_DATA = 'data/240621_young_time_course_binning.xls.csv'
F_YOUNG_TOUCH_DATA ='data/240621_young_message_report.xls.csv'

load files

In [49]:
#old_data = pd.read_csv('data/240621_old_time_course_binning.xls.csv')
#old_touch_data = pd.read_csv('data/240621_old_message_report.xls.csv')
#young_data = pd.read_csv('data/240621_young_time_course_binning.xls.csv')
#young_touch_data = pd.read_csv('data/240621_young_message_report.xls.csv')

old_data = pd.read_csv(F_OLD_DATA)
old_touch_data = pd.read_csv(F_OLD_TOUCH_DATA)
young_data = pd.read_csv(F_YOUNG_DATA)
young_touch_data = pd.read_csv(F_YOUNG_TOUCH_DATA)


not sure why its here

In [41]:
#z=old_touch_data.CURRENT_MSG_TEXT.unique()
#z.sort()
#z # [x for x in z if not x.startswith('TRACKER_TIME')]
#old_touch_data

In [40]:
#young_touch_data.loc[young_touch_data.CURRENT_MSG_TEXT.str.contains("TRIALID ")]
#young_touch_data.loc[(young_touch_data.TRIAL_INDEX == 5)]

rename columns names

In [9]:
for data in [old_data, old_touch_data, young_data, young_touch_data]:
    data.rename(columns={
        _PART:   PART,
        _TRIAL:  TRIAL,
        _TARGET: TARGET,
        _COMP:   COMP,
        _FILL1:  FILL1,
        _FILL2:  FILL2,
        _TIME:   TIME
    }, inplace=True)

fix the trial column to be the numbers from 'TRIAL_LABEL' column data 

In [10]:
old_data_1 = old_data.assign(**{TRIAL: lambda x: pd.to_numeric(x['TRIAL_LABEL'].str.strip('Trial: '))})
young_data_1 = young_data.assign(**{TRIAL: lambda x: pd.to_numeric(x['TRIAL_LABEL'].str.strip('Trial: '))})

#### get message report offset

 
 read the message report and extract TOUCH_TARGET time stamp and stimuli timestamp,
 
 calculate ofset for TOUCH_TARGET from mimimum of stimuli (when multiple stimuli exists)

In [11]:
od1 = old_touch_data
yd1 = young_touch_data
# od1['CURRENT_MSG_TEXT']
# TOUCH_TARGET ==> Selection_Target
# Stiimuli ==> Sound_Start
    # filter messages which are not TOUCH_TARGET nor stimuli ...
od2 = od1.loc[(od1['CURRENT_MSG_TEXT'] == 'Selection_Target') |
              (od1['CURRENT_MSG_TEXT'].str.contains('Sound_Start'))]

yd2 = yd1.loc[(yd1['CURRENT_MSG_TEXT'] == 'Selection_Target') |
              (yd1['CURRENT_MSG_TEXT'].str.contains('Sound_Start'))]


#od2

#od2[od2['CURRENT_MSG_TEXT'].str.contains('Sound_Start')].pivot_table(
#    index=[PART, TRIAL], values='CURRENT_MSG_TIME', aggfunc=min
#)

#od2


In [12]:

    # find which trials are missing entries of stimuli touch report bundle
od3 = od2.pivot_table(index=[PART], values=[
     'CURRENT_MSG_TEXT'], columns=[TRIAL], aggfunc=[len])
#od3

yd3 = yd2.pivot_table(index=[PART], values=[
     'CURRENT_MSG_TEXT'], columns=[TRIAL], aggfunc=[len])


In [ ]:
    # rename value from 'Stimuli: <NAME>' to just 'Stimuli'
#    _data4 = _data2.replace({'Stimuli:.*': 'Stimuli'}, regex=True)

In [13]:

    # pivot table to have columns for TOUCH_TARGET and Stimuli, have min value of CURRENT_MSG_TIME as the new value
od5 = od2.pivot_table(index=[PART, TRIAL],
                            columns=['CURRENT_MSG_TEXT'],
                            values='CURRENT_MSG_TIME', aggfunc=min)

yd5 = yd2.pivot_table(index=[PART, TRIAL],
                            columns=['CURRENT_MSG_TEXT'],
                            values='CURRENT_MSG_TIME', aggfunc=min)


#od5
    # find out missing TOUCH_TARGET trials
#od5[od5['Selection_Target'].isnull()]

In [15]:
    # get the int value instead of float
od5_1 = od5[~od5['Selection_Target'].isnull()].astype(int)

yd5_1 = yd5[~yd5['Selection_Target'].isnull()].astype(int)
#od5_1

In [16]:
   # fill Nan as 0, set type to int and calculate offset from Stimuli to TOUCH_TARGET
od6 = od5.fillna(0).astype(int).assign(
        TOUCH_FIXED=lambda x: x['Selection_Target'] - x['Sound_Start'])

yd6 = yd5.fillna(0).astype(int).assign(
        TOUCH_FIXED=lambda x: x['Selection_Target'] - x['Sound_Start'])


#od6


In [17]:
   # missing TOUCH_TARGET message report
od6[od6['TOUCH_FIXED'] < 0]
yd6[yd6['TOUCH_FIXED'] < 0]

CURRENT_MSG_TEXT   Selection_Target  Sound_Start  TOUCH_FIXED
Participant Trial                                            
104         1                     0         8539        -8539
109         13                    0         9886        -9886
            52                    0         7423        -7423
            65                    0         7841        -7841
110         10                    0        10388       -10388
            17                    0        10146       -10146
            41                    0        10091       -10091
            70                    0         8658        -8658
            75                    0         8840        -8840
113         18                    0        10549       -10549
118         17                    0        10762       -10762
            35                    0        10562       -10562
            59                    0         7084        -7084
125         7                     0         9584        -9584
127         9                     0         6391        -6391
128         1                     0        38956       -38956
133         29                    0        14916       -14916
135         16                    0         8253        -8253

In [18]:
    # rename only the indexs 
od7 = od6.rename_axis([PART, TRIAL])
yd7 = yd6.rename_axis([PART, TRIAL])
od7
yd7

CURRENT_MSG_TEXT   Selection_Target  Sound_Start  TOUCH_FIXED
Participant Trial                                            
101         1                115816       111088         4728
            2                 14438        11863         2575
            3                 13471        11763         1708
            4                 14967        11267         3700
            5                 14861        11062         3799
...                             ...          ...          ...
135         79                13804         9965         3839
            80                13224        10621         2603
            81                14225        10315         3910
            82                13176         9363         3813
            83                13098        10097         3001

[2546 rows x 3 columns]

get message report offset - finished

In [19]:

old_touch_data_1 = od6
young_touch_data_1 = yd6

merge the data with touch data

In [20]:
old_data_with_touch = pd.merge(old_data_1.set_index([PART, TRIAL]),
                               old_touch_data_1,
                               on=[PART, TRIAL])

young_data_with_touch = pd.merge(young_data_1.set_index([PART, TRIAL]),
                               young_touch_data_1,
                               on=[PART, TRIAL])

In [28]:
def plot_part_trial(data2, _p, _t):
#    (_p, _t) = (411, 9)
    p = data2.loc[_p].loc[_t].plot(x=TIME, y=TARGET)
    p2 = p.bar(x=data2.loc[_p].loc[_t].TOUCH_FIXED.values[0],
               height=10, width=20, color="red")
    p3 = p.bar(x=1500, height=10, width=20, color='magenta')
    p4 = p.bar(x=2700, height=10, width=20, color='black')
    p.set_ylim(-10, 100)
    p.set_title("Part: {}, Trial: {}, Type: {}".format(
        _p, _t, data2.loc[_p].loc[_t].index[0]), fontsize=TITLE_FONT_SIZE)
    p.legend(['Target', 'cutoff {}'.format(
        data2.loc[_p].loc[_t].TOUCH_FIXED.values[0]), 'qend: 1500', 'target: 2700'], loc='upper left',
        fontsize=LEGEND_FONT_SIZE)

In [35]:
#old_data_with_touch

In [36]:
#old_data_with_touch
#z=old_data_with_touch.set_index(TYPE)

#p = old_data_with_touch.loc[201].loc[5].plot(x=TIME, y=TARGET)

#plot_part_trial(z, 201, 5)

if we want to plot target for specific participant and trial

In [37]:
# not working ATM
#plot_part_trial(old_data_with_touch.set_index(PART, TRIAL, TYPE), 205, 5)

In [44]:
od1 = old_data_with_touch
yd1 = young_data_with_touch
od1
yd1

TRIAL_LABEL  BIN_INDEX  AVERAGE_BLINK_SAMPLE_COUNT_%  \
Participant Trial                                                        
101         1        Trial: 1          0                           1.0   
            1        Trial: 1          1                           1.0   
            1        Trial: 1          2                           1.0   
            1        Trial: 1          3                           1.0   
            1        Trial: 1          4                           1.0   
...                       ...        ...                           ...   
135         83      Trial: 83        146                           0.0   
            83      Trial: 83        147                           0.0   
            83      Trial: 83        148                           0.0   
            83      Trial: 83        149                           0.0   
            83      Trial: 83        150                           0.0   

                   AVERAGE_EXCLUDED_SAMPLE_COUNT_%  \
Participant Trial                                    
101         1                                  0.0   
            1                                  0.0   
            1                                  0.0   
            1                                  0.0   
            1                                  0.0   
...                                            ...   
135         83                                 0.0   
            83                                 0.0   
            83                                 0.0   
            83                                 0.0   
            83                                 0.0   

                   AVERAGE_IA_1_SAMPLE_COUNT_%  AVERAGE_IA_2_SAMPLE_COUNT_%  \
Participant Trial                                                             
101         1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
...                                        ...                          ...   
135         83                             1.0                          0.0   
            83                             1.0                          0.0   
            83                             1.0                          0.0   
            83                             1.0                          0.0   
            83                             1.0                          0.0   

                   AVERAGE_IA_3_SAMPLE_COUNT_%  AVERAGE_IA_4_SAMPLE_COUNT_%  \
Participant Trial                                                             
101         1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
...                                        ...                          ...   
135         83                             0.0                          0.0   
            83                             0.0                          0.0   
            83                             0.0                          0.0   
            83                             0.0                          0.0   
            83                             0.0                          0.0   

                   AVERAGE_IA_0_SAMPLE_COUNT_%  BIN_DURATION  ...  load  \
Participant Trial                                             ...         
101         1                              0.0            20  ...     4   
            1                              0.0            20  ...     4

In [51]:
yd1.columns

Index(['TRIAL_LABEL', 'BIN_INDEX', 'AVERAGE_BLINK_SAMPLE_COUNT_%',
       'AVERAGE_EXCLUDED_SAMPLE_COUNT_%', 'AVERAGE_IA_1_SAMPLE_COUNT_%',
       'AVERAGE_IA_2_SAMPLE_COUNT_%', 'AVERAGE_IA_3_SAMPLE_COUNT_%',
       'AVERAGE_IA_4_SAMPLE_COUNT_%', 'AVERAGE_IA_0_SAMPLE_COUNT_%',
       'BIN_DURATION', 'BIN_SAMPLE_COUNT', 'Time', 'IA_1_ID', 'IA_2_ID',
       'IA_3_ID', 'IA_4_ID', 'IA_0_ID', 'DigitsCorrect', 'Response',
       'digit_span_list', 'load', 'version', 'competitor', 'filler1',
       'filler2', 'target', 'type', 'Selection_Target', 'Sound_Start',
       'TOUCH_FIXED', 'Target'],
      dtype='object')

In [118]:
NEW_TARGET='TARGET__NEW'
od1['TOUCH_FIXED']
od1.loc[od1[TIME] >= od1['TOUCH_FIXED'], NEW_TARGET] = 1.0
yd1['TOUCH_FIXED']
yd1.loc[yd1[TIME] >= yd1['TOUCH_FIXED'], NEW_TARGET] = 1.0

# special override of values on'AVERAGE_IA_1_SAMPLE_COUNT_%'
od1.loc[od1[TIME] >= od1['TOUCH_FIXED'], 'AVERAGE_IA_1_SAMPLE_COUNT_%'] = 1.0
yd1.loc[yd1[TIME] >= yd1['TOUCH_FIXED'], 'AVERAGE_IA_1_SAMPLE_COUNT_%'] = 1.0


In [120]:
#od1.reset_index().to_csv(index=False)
od1

TRIAL_LABEL  BIN_INDEX  AVERAGE_BLINK_SAMPLE_COUNT_%  \
Participant Trial                                                        
201         1        Trial: 1          0                           0.0   
            1        Trial: 1          1                           0.0   
            1        Trial: 1          2                           0.0   
            1        Trial: 1          3                           0.0   
            1        Trial: 1          4                           0.0   
...                       ...        ...                           ...   
241         83      Trial: 83        140                           0.0   
            83      Trial: 83        141                           0.0   
            83      Trial: 83        142                           0.0   
            83      Trial: 83        143                           0.0   
            83      Trial: 83        144                           0.0   

                   AVERAGE_EXCLUDED_SAMPLE_COUNT_%  \
Participant Trial                                    
201         1                                  0.0   
            1                                  0.0   
            1                                  0.0   
            1                                  0.0   
            1                                  0.0   
...                                            ...   
241         83                                 0.0   
            83                                 0.0   
            83                                 0.0   
            83                                 0.0   
            83                                 0.0   

                   AVERAGE_IA_1_SAMPLE_COUNT_%  AVERAGE_IA_2_SAMPLE_COUNT_%  \
Participant Trial                                                             
201         1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
...                                        ...                          ...   
241         83                             1.0                          0.0   
            83                             1.0                          0.0   
            83                             1.0                          0.0   
            83                             1.0                          0.0   
            83                             1.0                          0.0   

                   AVERAGE_IA_3_SAMPLE_COUNT_%  AVERAGE_IA_4_SAMPLE_COUNT_%  \
Participant Trial                                                             
201         1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
            1                              0.0                          0.0   
...                                        ...                          ...   
241         83                             0.0                          0.0   
            83                             0.0                          0.0   
            83                             0.0                          0.0   
            83                             0.0                          0.0   
            83                             0.0                          0.0   

                   AVERAGE_IA_0_SAMPLE_COUNT_%  BIN_DURATION  ...  competitor  \
Participant Trial                                             ...               
201         1                              1.0            20  ...    ayin.jpg   
            1                              1.0       

now save file with padded target time

In [53]:
F_OLD_DATA_PADDED = F_OLD_DATA.split('.csv')[0] + '_padded.csv'
F_YOUNG_DATA_PADDED = F_YOUNG_DATA.split('.csv')[0] + '_padded.csv'
od1.reset_index().to_csv(F_OLD_DATA_PADDED, index=False)
yd1.reset_index().to_csv(F_YOUNG_DATA_PADDED, index=False)

In [79]:
#(select_target - sound_start ) per trial
od1.columns
od1.type
od11=od1[od1[TYPE] != 'p'].reset_index()
od11
od11[TYPE].unique()


array(['c', 'a', 'b'], dtype=object)

In [104]:
_q = od11.pivot_table(index=[TIME, PART, TRIAL],
                         values=[x for x in od1.columns if x.startswith('AVERAGE_IA_')] + [TYPE],
                         aggfunc={lambda x: x,
                     fill_value=-1,
                     dropna=False)

In [115]:
# find missing : -1 value 
_q[_q[_TARGET] == -1]
od11[od11[_TARGET] == 1.0]

,Participant,Trial,TRIAL_LABEL,BIN_INDEX,AVERAGE_BLINK_SAMPLE_COUNT_%,AVERAGE_EXCLUDED_SAMPLE_COUNT_%,AVERAGE_IA_1_SAMPLE_COUNT_%,AVERAGE_IA_2_SAMPLE_COUNT_%,AVERAGE_IA_3_SAMPLE_COUNT_%,AVERAGE_IA_4_SAMPLE_COUNT_%,...,version,competitor,filler1,filler2,target,type,Selection_Target,Sound_Start,TOUCH_FIXED,Target
81,201,4,Trial: 4,81,0.0,0.0,1.0,0.0,0.0,0.0,...,AH,aron.jpg,kumkum.jpg,tamar.jpg,shulxan.jpg,c,14753,10887,3866,NaN
82,201,4,Trial: 4,82,0.0,0.0,1.0,0.0,0.0,0.0,...,AH,aron.jpg,kumkum.jpg,tamar.jpg,shulxan.jpg,c,14753,10887,3866,NaN
83,201,4,Trial: 4,83,0.0,0.0,1.0,0.0,0.0,0.0,...,AH,aron.jpg,kumkum.jpg,tamar.jpg,shulxan.jpg,c,14753,10887,3866,NaN
84,201,4,Trial: 4,84,0.0,0.0,1.0,0.0,0.0,0.0,...,AH,aron.jpg,kumkum.jpg,tamar.jpg,shulxan.jpg,c,14753,10887,3866,NaN
85,201,4,Trial: 4,85,0.0,0.0,1.0,0.0,0.0,0.0,...,AH,aron.jpg,kumkum.jpg,tamar.jpg,shulxan.jpg,c,14753,10887,3866,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607921,241,83,Trial: 83,140,0.0,0.0,1.0,0.0,0.0,0.0,...,AH,kolav.jpg,masait.jpg,seret.jpg,nadneda.jpg,b,9089,6276,2813,NaN
607922,241,83,Trial: 83,141,0.0,0.0,1.0,0.0,0.0,0.0,...,AH,kolav.jpg,masait.jpg,seret.jpg,nadneda.jpg,b,9089,6276,2813,1.0
607923,241,83,Trial: 83,142,0.0,0.0,1.0,0.0,0.0,0.0,...,AH,kolav.jpg,masait.jpg,seret.jpg,nadneda.jpg,b,9089,6276,2813,1.0
607924,241,83,Trial: 83,143,0.0,0.0,1.0,0.0,0.0,0.0,...,AH,kolav.jpg,masait.jpg,seret.jpg,nadneda.jpg,b,9089,6276,2813,1.0


In [80]:
# od2 = od1[~[od1[TYPE] == 'p']]
od11.pivot_table(index=[TIME, PART, TRIAL],
                values=[x for x in od1.columns if x.startswith('AVERAGE_IA_')],
                aggfunc=lambda x: x)

AVERAGE_IA_0_SAMPLE_COUNT_%  \
Time  Participant Trial                                
0     201         4                              1.0   
                  5                              1.0   
                  6                              0.0   
                  7                              0.8   
                  8                              1.0   
...                                              ...   
18360 212         11                             1.0   
18380 212         11                             1.0   
18400 212         11                             1.0   
18420 212         11                             1.0   
18440 212         11                             1.0   

                         AVERAGE_IA_1_SAMPLE_COUNT_%  \
Time  Participant Trial                                
0     201         4                              0.0   
                  5                              0.0   
                  6                              0.0   
                  7                              0.2   
                  8                              0.0   
...                                              ...   
18360 212         11                             0.0   
18380 212         11                             0.0   
18400 212         11                             0.0   
18420 212         11                             0.0   
18440 212         11                             0.0   

                         AVERAGE_IA_2_SAMPLE_COUNT_%  \
Time  Participant Trial                                
0     201         4                              0.0   
                  5                              0.0   
                  6                              0.0   
                  7                              0.0   
                  8                              0.0   
...                                              ...   
18360 212         11                             0.0   
18380 212         11                             0.0   
18400 212         11                             0.0   
18420 212         11                             0.0   
18440 212         11                             0.0   

                         AVERAGE_IA_3_SAMPLE_COUNT_%  \
Time  Participant Trial                                
0     201         4                              0.0   
                  5                              0.0   
                  6                              1.0   
                  7                              0.0   
                  8                              0.0   
...                                              ...   
18360 212         11                             0.0   
18380 212         11                             0.0   
18400 212         11                             0.0   
18420 212         11                             0.0   
18440 212         11                             0.0   

                         AVERAGE_IA_4_SAMPLE_COUNT_%  
Time  Participant Trial                               
0     201         4                              0.0  
                  5                              0.0  
                  6                              0.0  
                  7                              0.0  
                  8                              0.0  
...                                              ...  
18360 212         11                             0.0  
18380 212         11                             0.0  
18400 212         11                             0.0  
18420 212         11                             0.0  
18440 212         11                             0.0  

[607926 rows x 5 columns]